In [1]:
import os
import urllib.request
import pandas as pd
import requests
import re
import datetime
from bs4 import BeautifulSoup 
from urllib.parse import quote # 아스키 코드 형식이 아닌 글자를 url 인코딩 해줌

In [2]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 시작날짜, 끝날짜 받기)
def MakeUrl(start_date, end_date):
    urls= [] # 링크를 넣을 리스트 생성
    # 검색 키워드
    k = ['치과', '치과비용', '충치', '치아교정', '잇몸치료', '치주염', '구내염', '치아미백', '시린이', '잇몸영양제']
    for keyword in k: 
        url = 'https://search.naver.com/search.naver?where=blog&query=' + quote(keyword) + f'&sm=tab_opt&nso=so:r,p:from{start_date}to{end_date}' # 시작날짜, 끝날짜
        urls.append(url)
    return urls 

In [3]:
# 제목, 링크, 발행날짜가 담긴 블로그 상위 10위까지 크롤링
def make_crawling(urls, start_date, end_date):
    # 로봇 아닙니다 인식하기 위해 임시로 넣는 코드
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

    # 엑셀 파일 생성
    writer=pd.ExcelWriter(f'C:\\Users\\User\\Desktop\\fproject\\매일블로그크롤링\\{start_date}~{end_date}.xlsx', engine='openpyxl')
    k = ['치과', '치과비용', '충치', '치아교정', '잇몸치료', '치주염', '구내염', '치아미백', '시린이', '잇몸영양제']
    
    # 블로그 제목, 링크, 발행날짜 뽑기
    idx = 0 # 키워드별로 엑셀 시트를 만들기 위해 인덱스 설정
    for url in urls: 
        response = requests.get(url, headers=headers)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')  

        title_list = []
        link_list = []
        pubdate_list = []

        for j in range(1, 11):
            items= soup.find_all('li', {'id':f'sp_blog_{j}'}) 
            # find_all로 가져오면 리스트형태로 가져옴
            # print(item)

            for item in items:
                print('\n') # 문자열 줄바꿈
                # 블로그 게시글 제목
                title = item.find('a',{'class':'api_txt_lines total_tit'}).get_text()
                title_list.append(title)
                print('제목 : ', title)
                
                # 링크 추출
                link = item.find('a',{'class':'api_txt_lines total_tit'})['href']
                link_list.append(link)
                print('링크 : ', link)    
                
                # 발행날짜 뽑기
                pubdate = item.find('span',{'class':'sub_time sub_txt'}).text.strip()
                pubdate = pubdate.rstrip('.') # 오른쪽 끝단의 '.' 삭제
                pubdate = pubdate.replace('.', '-')

                # 3시간 전, 4일 전, 어제 따로 처리
                now = datetime.datetime.now() # 오늘 날짜 불러오기
                if '시간 전' in pubdate:
                    pubdate = now.strftime('%Y-%m-%d')
                    pubdate_list.append(pubdate)
                elif '일 전' in pubdate:
                    temp = int(pubdate[0:1])
                    temp = datetime.timedelta(days=temp)
                    pubdate = datetime.datetime.now() - temp
                    pubdate = pubdate.strftime('%Y-%m-%d')
                    pubdate_list.append(pubdate)
                elif '어제' in pubdate:
                    temp = datetime.timedelta(days=1)
                    pubdate = datetime.datetime.now() - temp
                    pubdate = pubdate.strftime('%Y-%m-%d')
                    pubdate_list.append(pubdate)
                else:
                    pubdate_list.append(pubdate)
                print('발행일자 : ',pubdate)

    # 제목, 링크, 발행일자를 Dataframe으로 만들기 
        df = pd.DataFrame({'제목':title_list, '링크':link_list, '발행일자':pubdate_list})
    # 키워드당 하나의 시트로 만들어서 엑셀 파일 하나에 저장 
        df.to_excel(writer, sheet_name=f'{k[idx]}')
        idx += 1
        writer.save()
    return df

### 함수 전체 호출

In [ ]:
# 메인 함수 : 함수 한꺼번에 실행
if __name__=='__main__':
    # 날짜 받기
    start_date = input('검색 시작 년도(year)를 입력해주세요 : ex)20220505')
    end_date = input('검색 끌 일(date)을 입력해주세요 : ex)20220519')
    urls = MakeUrl(start_date, end_date)
    make_crawling(urls, start_date, end_date)
